In [10]:
import pandas as pd
import matplotlib as mpl
pd.Series.__unicode__ = pd.Series.to_string

In [11]:
df=pd.read_csv("./census2011.csv")

In [12]:
df

,Person ID,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,Approximated Social Grade
0,7394816,E12000001,H,2,1,2,6,2,2,1,2,1,2,5,8,2,-9,4
1,7394745,E12000001,H,5,1,1,4,1,2,1,1,1,2,1,8,6,4,3
2,7395066,E12000001,H,3,1,2,4,1,2,1,1,1,1,1,6,11,3,4
3,7395329,E12000001,H,3,1,2,2,1,2,1,2,1,2,1,7,7,3,2
4,7394712,E12000001,H,3,1,1,5,4,2,1,1,1,2,1,1,4,3,2
5,7394750,E12000001,H,2,1,1,6,2,2,1,2,1,1,1,9,2,3,3
6,7394871,E12000001,H,5,1,2,4,3,2,1,2,1,1,1,6,11,2,3
7,7394832,E12000001,H,3,1,2,1,1,2,1,2,1,1,-9,-9,-9,-9,-9
8,7394719,E12000001,H,2,1,1,7,2,2,1,1,1,2,5,8,2,-9,2
9,7394840,E12000001,H,1,1,2,6,4,2,1,3,1,2,5,9,5,-9,4


In [13]:
len(df)

569741

In [14]:
print("All Data Types:\n" + str(df.dtypes))

All Data Types:
Person ID                     int64
Region                       object
Residence Type               object
Family Composition            int64
Population Base               int64
Sex                           int64
Age                           int64
Marital Status                int64
Student                       int64
Country of Birth              int64
Health                        int64
Ethnic Group                  int64
Religion                      int64
Economic Activity             int64
Occupation                    int64
Industry                      int64
Hours worked per week         int64
Approximated Social Grade     int64


In [15]:
#counting the number of records
print("number of records = " + str(df['Person ID'].count()))

number of records = 569741


In [7]:
df['Residence Type'].value_counts()

H    559087
C     10654
Name: Residence Type, dtype: int64

In [9]:
for column in df:
    if (column != "Person ID"):
        print(str(column)+ " Counts\n" +str(df[column].value_counts()) + "\n")

Region Counts
E12000008    88084
E12000007    83582
E12000002    71436
E12000006    59411
E12000005    56875
E12000009    53774
E12000003    53471
E12000004    45782
W92000004    30977
E12000001    26349
Name: Region, dtype: int64

Residence Type Counts
H    559087
C     10654
Name: Residence Type, dtype: int64

Family Composition Counts
 2    300962
 1     96690
 3     72641
 5     64519
-9     18851
 4      9848
 6      6230
Name: Family Composition, dtype: int64

Population Base Counts
1    561040
2      6730
3      1971
Name: Population Base, dtype: int64

Sex Counts
2    289172
1    280569
Name: Sex, dtype: int64

Age Counts
1    106832
4     78641
5     77388
3     75948
2     72785
6     65666
7     48777
8     43704
Name: Age, dtype: int64

Marital Status Counts
1    270999
2    214180
4     40713
5     31898
3     11951
Name: Marital Status, dtype: int64

Student Counts
2    443204
1    126537
Name: Student, dtype: int64

Country of Birth Counts
 1    485645
 2     77292
-9   